# 5.1 ランダムフォレストとは

本ファイルは、5.1節の実装です。

機械学習モデルのランダムフォレストを解説・実装します。
決定木の分類、決定木の回帰、ランダムフォレストの分類、ランダムフォレストの回帰を実施します

## プログラム実行前の設定など

In [0]:
# 乱数のシードを設定
import random
import numpy as np

np.random.seed(1234)
random.seed(1234)


In [0]:
# 使用するパッケージ（ライブラリと関数）を定義
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# SciPy 平均0、分散1に正規化（標準化）関数
import scipy.stats

# シグモイド関数をimport
from scipy.special import expit

# その他
import pandas as pd


## データの作成

In [0]:
# データ数
num_data = 200

# 年齢
x_1 = randint(15, 76, num_data)  # 15から75歳の一様乱数

# 性別（0を女性、1を男性とします）
x_2 = randint(0, 2, num_data)  # 0か1の乱数


## テレビCMを見たかどうか

In [0]:
# ノイズの生成
e_z = randn(num_data)

# シグモイド関数に入れる部分
z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z

# シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうかの変数（0は見ていない、1は見た）
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)


## 購入量Yを作成

In [0]:
# ノイズの生成
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y


## データをまとめた表を作成し、平均値を比べる

In [6]:
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })

df.head()  # 先頭を表示


,年齢,性別,CMを見た,購入量
0,62,0,1.0,24.464285
1,34,0,0.0,45.693411
2,53,1,1.0,64.998281
3,68,1,1.0,47.186898
4,27,1,0.0,100.114260


In [7]:
# 平均値を比べる

print(df[df["CMを見た"] == 1.0].mean())
print("--------")
print(df[df["CMを見た"] == 0.0].mean())


年齢       55.836066
性別        0.483607
CMを見た     1.000000
購入量      49.711478
dtype: float64
--------
年齢       32.141026
性別        0.692308
CMを見た     0.000000
購入量      68.827143
dtype: float64


## 決定木で分類

決定木でCMを見たかどうかを分類予測するモデルを構築します

In [8]:
# scikit-learnから決定木の分類をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
from sklearn.tree import DecisionTreeClassifier

# データを訓練と検証に分割する
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.model_selection import train_test_split


# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Z = df["CMを見た"]

# データを訓練と検証に分割
X_train, X_val, Z_train, Z_val = train_test_split(
    X, Z, train_size=0.6, random_state=0)

# 学習と性能確認
clf = DecisionTreeClassifier(max_depth=1, random_state=0)
clf.fit(X_train, Z_train)
print("深さ1の性能：", clf.score(X_val, Z_val))  # 正解率を表示

# 学習と性能確認
clf = DecisionTreeClassifier(max_depth=2, random_state=0)
clf.fit(X_train, Z_train)
print("深さ2の性能：", clf.score(X_val, Z_val))  # 正解率を表示

# 学習と性能確認
clf = DecisionTreeClassifier(max_depth=3, random_state=0)
clf.fit(X_train, Z_train)
print("深さ3の性能：", clf.score(X_val, Z_val))  # 正解率を表示


深さ1の性能： 0.85
深さ2の性能： 0.85
深さ3の性能： 0.825


## 決定木で回帰

決定木で購入量を回帰予測するモデルを構築します

In [9]:
# scikit-learnから決定木の回帰をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

# データを訓練と検証に分割する
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.model_selection import train_test_split


# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Y = df["購入量"]

# データを訓練と検証に分割
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, train_size=0.6, random_state=0)

# 学習と性能確認
reg = DecisionTreeRegressor(max_depth=2, random_state=0)
reg = reg.fit(X_train, Y_train)
print("深さ2の性能：", reg.score(X_val, Y_val))  # 決定係数R2を表示

# 学習と性能確認
reg = DecisionTreeRegressor(max_depth=3, random_state=0)
reg = reg.fit(X_train, Y_train)
print("深さ3の性能：", reg.score(X_val, Y_val))  # 決定係数R2を表示

# 学習と性能確認
reg = DecisionTreeRegressor(max_depth=4, random_state=0)
reg = reg.fit(X_train, Y_train)
print("深さ4の性能：", reg.score(X_val, Y_val))  # 決定係数R2を表示


深さ2の性能： 0.7257496664596153
深さ3の性能： 0.7399348963931736
深さ4の性能： 0.7165539691159019


## ランダムフォレストで分類

ランダムフォレストでCMを見たかどうかを分類予測するモデルを構築します

In [10]:
# scikit-learnからランダムフォレストの分類をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html?highlight=randomforest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Z = df["CMを見た"]

# データを訓練と検証に分割
X_train, X_val, Z_train, Z_val = train_test_split(
    X, Z, train_size=0.6, random_state=0)

# 学習と性能確認
clf = RandomForestClassifier(max_depth=1, random_state=0)
clf.fit(X_train, Z_train)
print("深さ1の性能：", clf.score(X_val, Z_val))  # 正解率を表示

# 学習と性能確認
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, Z_train)
print("深さ2の性能：", clf.score(X_val, Z_val))  # 正解率を表示

# 学習と性能確認
clf = RandomForestClassifier(max_depth=3, random_state=0)
clf.fit(X_train, Z_train)
print("深さ3の性能：", clf.score(X_val, Z_val))  # 正解率を表示


深さ1の性能： 0.775
深さ2の性能： 0.85
深さ3の性能： 0.825


## ランダムフォレストで回帰

ランダムフォレストで購入量を回帰予測するモデルを構築します

In [11]:
# scikit-learnから決定木の回帰をimport
# https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html?highlight=randomforest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


# 説明変数
X = df[["年齢", "性別"]]

# 被説明変数（目的変数）
Y = df["購入量"]

# データを訓練と検証に分割
X_train, X_val, Y_train, Y_val = train_test_split(
    X, Y, train_size=0.6, random_state=0)

# 学習と性能確認
reg = RandomForestRegressor(max_depth=2, random_state=0)
reg = reg.fit(X_train, Y_train)
print("深さ2の性能：", reg.score(X_val, Y_val))  # 決定係数R2を表示

# 学習と性能確認
reg = RandomForestRegressor(max_depth=3, random_state=0)
reg = reg.fit(X_train, Y_train)
print("深さ3の性能：", reg.score(X_val, Y_val))  # 決定係数R2を表示

# 学習と性能確認
reg = RandomForestRegressor(max_depth=4, random_state=0)
reg = reg.fit(X_train, Y_train)
print("深さ4の性能：", reg.score(X_val, Y_val))  # 決定係数R2を表示


深さ2の性能： 0.7618786062003249
深さ3の性能： 0.7810610687821996
深さ4の性能： 0.7655149049335735


以上